In [1]:
#import libraries 
import pandas as pd
import requests
import json
import tweepy
import re
import csv
import asyncio
import time
import openpyxl
import nest_asyncio
import tkinter as tk
from tkinter import filedialog
from telethon.sync import TelegramClient
from openpyxl import load_workbook
from openpyxl.styles import NamedStyle
from bs4 import BeautifulSoup

In [ ]:
#X's API Initialization
Bearer_Token = "AAAAAAAAAAAAAAAAAAAAAP2%2BzgEAAAAAcesSwufQuNkNph4oM3%2BPI1tQoiM%3DfZnoWNQJpEN1OXvrAKglj5PZYGaHtbQWedIjjBgOmIhkTxmTCb"
API_Key = "eXHKcdVzjeLi8CLbdeDZ1MWpy"
X_KeySecret = "hoyHQeyOtbLSAWQgnueNpd2XASOycVUo1utZ8sThdZ9SsKxRJc"
X_Token = "807562185681633282-pQEABmamoUon6wv2FG4aotgGpzTleTI"
X_AccessToken = "bV0dmHTIWkZ0mkfazL1Mqp5Uc9jQO4N89ybNp4k9S9jHv"

#Authentication with OAuth
auth = tweepy.OAuth1UserHandler(API_Key, X_KeySecret,X_Token,X_AccessToken)
api = tweepy.API(auth)
user = tweepy.Client(bearer_token=Bearer_Token)

# Check authentication
try:
    api.verify_credentials()
    print("Twitter API authentication successful!")
except Exception as e:
    print(f"Authentication error: {e}")

In [ ]:
#Telegram credentials
API_ID = "26137609"
API_Hash = "837e683dbc20e9b13f368688f00954ed"
Phone_number = "+2348027343812"

client = TelegramClient("Scrape", API_ID, API_Hash)
async def main():
    await client.start()
    me = await client.get_me()  # Fetch user info to confirm login
    print(f"✅ Authentication successful! Logged in as: {me.username} ({me.id})")

try:
    asyncio.get_running_loop()
    print("Running inside an event loop, using 'await main()' instead...")
    await main()
except RuntimeError:
    print("Running normally with asyncio.run()")
    asyncio.run(main())

In [3]:
#Open a file dialog
source = tk.Tk()
source.withdraw() #Hide the root window
source.update()

file_path = filedialog.askopenfilename(title="Select an Excel file", filetypes=[("Excel files", "*.xlsx")])
if file_path:
    print(f"File selected: {file_path}")
else:
    print("No file selected. Exiting...")
    exit()

# Load the Excel file
wb = load_workbook(file_path)
default_style = NamedStyle(name="default")
wb.add_named_style(default_style)

File selected: C:/Users/FAVOUR ANAZODO/Documents/Python Projects/SHILL SQUAD.xlsx


In [ ]:
# Function to extract hyperlinks from a cell
def hidden_hyperlink(cell):
    if cell.hyperlink:
        return cell.hyperlink.target  # Extract hyperlink if present
    return None

#Link Counts
X_Count = 0
telegram_count = 0
debank_count = 0
other_count

# === TWITTER (X) API SECTION ===
def fetch_tweets(tweet_ids):
    """
    Fetch engagement data for multiple tweets in a batch request.
    """
        
    results = {}
    batch_size = 100  # Twitter API allows max 100 tweet IDs per request

    for i in range(0, len(tweet_ids), batch_size):
        batch = tweet_ids[i:i + batch_size]  # Split into chunks of 100
        try:
            response = user.get_tweets(ids=batch, tweet_fields=["public_metrics"])
            
            if response.data:
                for tweet in response.data:
                    metrics = tweet.public_metrics
                    results[str(tweet.id)] = {
                        "likes": metrics["like_count"],
                        "retweets": metrics["retweet_count"],
                        "replies": metrics["reply_count"],
                    }
            
            # ✅ Respect Rate Limits (Sleep for 1 second between batches)
            time.sleep(1)
        
        except tweepy.TooManyRequests:
            print("⚠️ Rate limit exceeded. Sleeping for 15 minutes...")
            time.sleep(15 * 60)  # Wait for 15 minutes before retrying
        
        except Exception as e:
            print(f"❌ Error fetching tweets: {e}")
    
    return results  # Returns {tweet_id: {likes, retweets, replies}}

def X_data(url):
    """Extract Tweet ID and fetch engagement data."""
    match = re.search(r'status/(\d+)', url)
    return match.group(1) if match else None

# === TELEGRAM API SECTION ===
async def fetch_telegram_data(channel_username, message_id):
    """Fetch views, reactions, and forwards for a given Telegram post."""
    try:
        message = await client.get_messages(channel_username, ids=message_id)
        return {
            "views": message.views if message.views else 0,
            "reactions": message.reactions.to_dict() if message.reactions else {},
            "forwards": message.forwards if message.forwards else 0
        }
    except Exception as e:
        return f"Error fetching Telegram data: {e}"

def telegram_data(url):
    """Extract Telegram message info and fetch engagement data asynchronously."""
    match = re.search(r"t\.me/([^/]+)/(\d+)", url)
    if match:
        channel_username = match.group(1)
        message_id = int(match.group(2))

        try:
            loop = asyncio.get_running_loop()  # ✅ Get the running event loop
            return asyncio.ensure_future(fetch_telegram_data(channel_username, message_id))  # ✅ Schedule task
        except RuntimeError:
            return asyncio.run(fetch_telegram_data(channel_username, message_id))  # ✅ If no loop, start one

    return "Invalid Telegram URL"


# === SCRAPING PROCESS ===
# Store tweet IDs and async Telegram tasks
tweet_ids = []
telegram_tasks = []
all_data = []

for sheet_name in wb.sheetnames:
    sheet = wb[sheet_name]
    
    # Process each row
    for row in sheet.iter_rows(min_row=2, values_only=False):  
        first_col_value = row[0].value
        for cell in row:
            url = hidden_hyperlink(cell)
            if url:
                if "x.com" in url:
                    tweet_id = X_data(url)
                    if tweet_id:
                        tweet_ids.append(tweet_id)
                        all_data.append([first_col_value, url, tweet_id])  # Store tweet_id for batch processing
                elif "t.me" in url or "telegram.me" in url:
                    task = telegram_data(url)
                    if task:
                        telegram_tasks.append(task)
                        all_data.append([first_col_value, url, task])  # Store task for async execution
                else:
                    all_data.append([first_col_value, url, "Unsupported website"])

# === FETCH DATA IN BATCHES ===
# 1️⃣ Fetch Twitter Data in Batch
if tweet_ids:
    twitter_results = fetch_tweets(tweet_ids)  # Call the function directly, do NOT wrap in asyncio task
    for row in all_data:
        if isinstance(row[2], str) and row[2] in twitter_results:  # ✅ Ensure it's a valid string
            engagement = twitter_results[row[2]]  # ✅ Get the dictionary
            row[2] = f"Likes: {engagement['likes']}, Retweets: {engagement['retweets']}, Replies: {engagement['replies']}"

# 2️⃣ Fetch Telegram Data in Parallel
if telegram_tasks:
    telegram_results = await asyncio.gather(*telegram_tasks)
    task_index = 0
    for row in all_data:
        if isinstance(row[2], asyncio.Future):  # Identify async tasks
            row[2] = telegram_results[task_index]  # Replace task with actual results
            task_index += 1

# === SAVE TO CSV ===
output_csv = "scraped_data.csv"
with open(output_csv, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["Name", "Link", "Scraped Data"])
    writer.writerows(all_data)

print(f"✅ Scraping completed. Data saved in {output_csv}")

# Disconnect Telegram Client
client.disconnect()

#Run Analysis
#Load CSV
df = pd.read_csv("scraped_data.csv")

In [ ]:
# Load the Excel file
excel_workbook = "SHILL SQUAD.xlsx"
wb = load_workbook(excel_workbook)
default_style = NamedStyle(name="default")
wb.add_named_style(default_style)

# Function to extract hyperlinks from a cell
def hidden_hyperlink(cell):
    if cell.hyperlink:
        return cell.hyperlink.target  # Extract hyperlink if present
    return None  # Return None if no hyperlink

# Function to batch fetch tweet engagement data
def fetch_tweet_engagement(tweet_ids):
    try:
        response = user.lookup_statuses(tweet_ids, tweet_mode="extended")  # Correct method
        results = {}

        for tweet in tweets:
            results[tweet.id] = {
                "likes": tweet.favorite_count,
                "retweets": tweet.retweet_count,
                "replies": tweet.reply_count if hasattr(tweet, "reply_count") else 0,
            }
        return results    # Ensure we return a dictionary
    except tweepy.TooManyRequests as e:
        print(f"Rate limit exceeded. Sleeping for {e.retry_after} seconds...")
        time.sleep(e.retry_after)  # Sleep only for required time
        return fetch_tweet_engagement(tweet_ids)  # Retry after sleep
    except Exception as e:
        print(f"Error fetching tweets: {e}")
        return {}    # Return an empty dictionary instead of a string

def X_data(url):
    match = re.search(r'status/(\d+)', url)
    if match:
        tweet_id = match.group(1)
        try:
            time.sleep(3)  # Wait 3 seconds between requests
            tweet = user.get_tweet(tweet_id, tweet_fields=["public_metrics"])
            if tweet.data:
                metrics = tweet.data["public_metrics"]
                return {
                    "likes": metrics["like_count"],
                    "retweets": metrics["retweet_count"],
                    "replies": metrics["reply_count"],
                }
            else:
                return "Tweet not found"
        except Exception as e:
            return f"Error fetching tweet: {e}"
    return "Invalid URL"
    
# Async function to fetch Telegram engagement data
async def fetch_telegram_data(channel_username, message_id):
    return await client.get_messages(channel_username, ids=message_id)

# Function to get Telegram data
def telegram_data(url):
    match = re.search(r"t\.me/([^/]+)/(\d+)", url)  # Extract channel & message ID
    if match:
        channel_username = match.group(1)
        message_id = int(match.group(2))

        try:
            loop = asyncio.get_event_loop()
            if loop.is_running():
                # If event loop is already running, create a new task
                message = asyncio.ensure_future(fetch_telegram_data(channel_username, message_id))
                return message.result()  # Wait for the result
            else:
                # If no event loop, run normally
                return loop.run_until_complete(fetch_telegram_data(channel_username, message_id))
        except Exception as e:
            return f"Error fetching Telegram data: {e}"
    return "Invalid Telegram URL"

# Function to determine scraping method
def scrape_data(url):
    if "x.com" in url:
        return X_data(url)
    elif "t.me" in url or "telegram.me" in url:
        return telegram_data(url)
    else:
        return "Unsupported website"

# Process each sheet in the Excel file
all_data = []
for sheet_name in wb.sheetnames:
    sheet = wb[sheet_name]
    
    # Extract headers (first row)
    headers = [cell.value for cell in sheet[1]]
    
    # Process each row
    for row in sheet.iter_rows(min_row=2, values_only=False):  # Skip headers
        row_data = [cell.value for cell in row]  # Extract normal values
        first_col_value = row[0].value  # Store first column value
        
        # Find and process hyperlinks
        for cell in row:
            url = hidden_hyperlink(cell)
            if url:
                scraped_info = scrape_data(url)
                all_data.append([first_col_value, url, scraped_info])  # Maintain first column value

# Write the output to a CSV file
output_csv = "scraped_data.csv"
with open(output_csv, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["First Column", "Link", "Scraped Data"])  # Header
    writer.writerows(all_data)

print(f"Scraping completed. Data saved in {output_csv}")

# Disconnect Telegram Client
client.disconnect()


In [ ]:
# Function to extract hyperlinks from excel cell
def hidden_hyperlink(sheet):
    links = []
    for row in sheet.iter_rows():
        for cell in row:
            if cell.hyperlink:
                links.append((cell.row, cell.column, cell.hyperlink.target))
    return links

# Function to scrape Twitter data
def X_data(url):
    match = re.search(r'status/(\d+)', url)
    if match:
        tweet_id = match.group(1)
        try:
            tweet = X.show_status(id=tweet_id)
            return tweet  # Returns full tweet data
        except Exception as e:
            print(f"Error fetching tweet: {e}")
    return None

# Function to scrape Telegram data
def scrape_telegram(url):
    try:
        response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
        soup = BeautifulSoup(response.text, "html.parser")
        # Example: Extract title of Telegram page (modify as needed)
        title = soup.find("title")
        return title.text if title else "No data found"
    except Exception as e:
        return f"Error scraping Telegram: {e}"


def process_excel(file_path):
#load excel sheets
    workbook = openpyxl.load_workbook(file_path)
    output_data = []
#parse through sheets    
    for sheet_name in workbook.sheetnames:
        sheet = workbook[sheet_name]
        hyperlinks = hidden_hyperlinks(sheet)
#extract based on link        
        for row, col, url in hyperlinks:
            if "twitter.com" in url:
                tweet_data = X_data(url)
                if tweet_data:
                    output_data.append([sheet.cell(row, 1).value, sheet.cell(1, col).value, tweet_data])
            elif "t.me" in url:
                telegram_data = Telegram_data(url)
                if telegram_data:
                    output_data.append([sheet.cell(row, 1).value, sheet.cell(1, col).value, telegram_data])
            else:
                output_data.append([sheet.cell(row, 1).value, sheet.cell(1, col).value, f"Other link: {url}"])
    
    df = pd.DataFrame(output_data, columns=["First Column", "First Row", "Extracted Data"])
    df.to_csv("output.csv", index=False)
    print("Data extraction complete. Saved to output.csv")

# Run process
process_excel("your_excel_file.xlsx")



    


In [ ]:

# Function to determine scraping method
def scrape_data(url):
    if "twitter.com" in url:
        return scrape_twitter(url)
    elif "t.me" in url or "telegram.me" in url:
        return scrape_telegram(url)
    else:
        return "Unsupported website"

# Process each sheet in the Excel file
all_data = []
for sheet_name in wb.sheetnames:
    sheet = wb[sheet_name]
    
    # Extract headers (first row)
    headers = [cell.value for cell in sheet[1]]
    
    # Process each row
    for row in sheet.iter_rows(min_row=2, values_only=False):  # Skip headers
        row_data = [cell.value for cell in row]  # Extract normal values
        first_col_value = row[0].value  # Store first column value
        
        # Find and process hyperlinks
        for cell in row:
            url = get_hyperlink(cell)
            if url:
                scraped_info = scrape_data(url)
                all_data.append([first_col_value, url, scraped_info])  # Maintain first column value

# Write the output to a CSV file
output_csv = "scraped_data.csv"
with open(output_csv, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["First Column", "Link", "Scraped Data"])  # Header
    writer.writerows(all_data)

print(f"Scraping completed. Data saved in {output_csv}")


In [ ]:
nest_asyncio.apply()

#Telegram credentials
API_ID = "26137609"
API_Hash = "837e683dbc20e9b13f368688f00954ed"
Phone_number = "+2348027343812"

#Authentication


async def check_auth():
    async with TelegramClient("Scrape", API_ID, API_Hash) as client:
        me = await client.get_me()  # Get account info
        if me:
            print(f"Authentication successful! Logged in as: {me.username} ({me.id})")
        else:
            print("Authentication failed!")

asyncio.run(check_auth())